# Calculate Partial Derivatives of Phi to O2,T,S at O2=O2_bar, T=T_bar, S=S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']

### the partial derivatives

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_dPhidO2_dPhidT_dPhidS_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)

## dT, dS, dO2, dAOU

In [21]:
#varr = 'O2'
#varr = 'AOU'
#varr = 'TEMP'
varr = 'SALT'

In [22]:
filename = f'{path}/DPLE-driftcorrected_d{varr}_ens_mean_monthly.nc'
ds0 = xr.open_dataset(filename)
dvar = ds0[f'd{varr}']

## calculate Phi components

In [6]:
import traits_db
df = traits_db.df_mi
df.head()

,Species,Phylum,Ao,Ac,Eo
0,Acanthephyra acutifrons,Crustacea,26.809651,13.364732,-0.0278
1,Acanthephyra curtirostris,Crustacea,25.575448,14.364194,0.0380
2,Acanthephyra purpurea,Crustacea,23.310023,7.460879,0.1320
3,Acanthephyra smithi,Crustacea,28.985507,17.657939,-0.0301
5,Acipenser brevirostrum,Chordata,22.123894,4.901716,0.3626


In [7]:
Ac_dist = np.percentile(df.Ac, [50])
#Ao_dist = np.percentile(df.Ao, [25, 50, 75])
Eo_dist = np.percentile(df.Eo, [25, 50, 75])
print(Ac_dist)
#print(Ao_dist)
print(Eo_dist)

[7.34832416]
[0.1985 0.3389 0.5625]


In [23]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product

dsg = xr.Dataset()
for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    if varr == 'O2':
        dsg[f'Phi_{varr}_{nrow*i+j}'] =  ds[f'dPhidO2_{nrow*i+j}'] * dvar #dEodT=0.022
        print('Phi O2 done')
    elif varr == 'AOU':
        dsg[f'Phi_{varr}_{nrow*i+j}'] = -ds[f'dPhidO2_{nrow*i+j}'] * dvar
        print('Phi AOU done')
    elif varr == 'TEMP':
        dsg[f'Phi_{varr}_{nrow*i+j}'] = ds[f'dPhidT_{nrow*i+j}'] * dvar  #dEodT=0.022
        print('Phi T done')
    elif varr == 'SALT':
        dsg[f'Phi_{varr}_{nrow*i+j}'] = ds[f'dPhidS_{nrow*i+j}'] * dvar  #dEodT=0.022
        print('Phi S done')

7.3483241595629805 0.1985
Phi S done
7.3483241595629805 0.3389
Phi S done
7.3483241595629805 0.5625
Phi S done
CPU times: user 42.2 s, sys: 3min 21s, total: 4min 3s
Wall time: 4min 23s


In [24]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsg.load()

<xarray.Dataset>
Dimensions:     (z_t: 35, nlat: 140, nlon: 195, Y: 64, L: 122)
Coordinates:
  * z_t         (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT        (nlat, nlon) float64 14.96 14.96 14.97 ... 70.43 70.42 70.42
    ULONG       (nlat, nlon) float64 101.7 102.9 104.0 ... 318.9 319.5 320.0
    ULAT        (nlat, nlon) float64 15.13 15.13 15.14 ... 70.64 70.63 70.63
    TLONG       (nlat, nlon) float64 101.2 102.3 103.4 ... 318.6 319.2 319.7
  * Y           (Y) int64 1954 1955 1956 1957 1958 ... 2013 2014 2015 2016 2017
  * L           (L) int64 1 2 3 4 5 6 7 8 9 ... 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_SALT_0  (L, z_t, nlat, nlon, Y) float64 nan nan nan nan ... nan nan nan
    Phi_SALT_1  (L, z_t, nlat, nlon, Y) float64 nan nan nan nan ... nan nan nan
    Phi_SALT_2  (L, z_t, nlat, nlon, Y) float64 nan nan nan nan ... nan nan nan

In [25]:
dsg.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_components_{varr}_ens_mean_monthly.nc', mode='w')